In [31]:
# Import libs
import pandas as pd

In [32]:
# Read in csv for all universities
df = pd.read_csv('all_posts.csv')
df.shape

(2996, 7)

Social media tends to have a lot of memes/non-sensical discussion. For this project it would be valuable to filter out these types of posts and focus on posts that are more likely to have criticisms about the university.

In [33]:
# Want to see post flairs, and subsequently filter for non-shitposts
df['flair'].unique()

array(['Transfers', 'Discussion', 'Humour', 'Serious', 'News', 'Other',
       'Rant', 'Health', 'Clubs/Sports', 'Social', 'Event', 'Courses',
       'Waterloo #173', 'Academics', 'Advice', 'Confirmed', 'Meta', nan,
       'Shitpost', 'Lost & Found', 'Question', 'Politics', 'Life Advice',
       'Admissions', 'Finances', 'Programs', 'ACORN/Quercus/Outlook',
       'Free Speech', 'Waterloo #201–250', 'UTM/UTSC',
       "I'm in High School", 'Jobs', 'Photography & Art',
       'Pho(ur seasons)tography & Art', '@ SFU (Exception)', 'Megathread',
       'Congrats, you made it!', 'Photography &amp; Art',
       'Humour - Satire', 'SFU = Studying For UBC', 'Prose', '🍁',
       '100% super duper confirmed by the r/byssey', '🔥🔥🔥',
       'Ghost-type Humour', 'Unverified', 'Lost Dog', '🎉🎉🎉',
       'Read Comments Section for full context', 'Missing Person', 'F',
       'Spicy', 'HQ Post', 'Spicy Meme', 'We did it, reddit!',
       'HQ shitpost', 'Certified Dank', 'shitpost', 'Political',
       

In [34]:
# Most popular flairs
df['flair'].value_counts()[:10]

flair
Humour               961
Discussion           223
Other                159
Photography & Art    116
shitpost              73
News                  49
Academics             28
Advice                22
Courses               22
HQ Post               21
Name: count, dtype: int64

In [35]:
df = df[df['flair'].isin(['Discussion', 'News', 'Academics', 'Advice', 'Courses', 'HQ Post'])]
df.shape

(365, 7)

# Cleaning Data

In [36]:
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Init stopwords
stop_words = set(stopwords.words("english"))

def clean_text(df):
    '''
    This function does 3 things

    1. Combines text from title, description and comments
    2. Normalizes university names
    3. Tokenizes and lemmatizes text and removes stopwords + reddit specific words

    '''
    remove_words = ['removed', 'deleted', '[ removed by reddit ]', '[deleted]']
    # custom_stopwords = {'student', 'course', 'people', 'get', 'like', 'time', 'year', 'would'}
    # Combine text from title, description and comments
    def combine_text(text): 

        if isinstance(text, list):
            return ' '.join([word for word in text if isinstance(word, str) and word.lower() not in remove_words])
        elif isinstance(text, str): 
            # Filter out posts removed by reddit
            if re.search(r"\[?\s*removed by reddit.*?\]?", text, flags=re.IGNORECASE):
                return ''
            return text
        return ''

    
    
    # Normalize university names
    def normalize_university(text):
        # Regex to normalize uni names
        text = re.sub(r'u\sof\st', 'uoft', text)
        text = re.sub(r'university of toronto', 'uoft', text)
        text = re.sub(r'university of british columbia', 'ubc', text)
        return text

    # Tokenize + lematize + remove stopwords
    def tokenize_and_lemmatize(text): 

        # Tokenize
        tokens = word_tokenize(text)

        # Filter for strings
        words = [word for word in tokens if word.isalpha()]

        # Remove stopwords
        words = [word for word in words if word not in stop_words]
        
        # Lemmatize
        lemmatizer = WordNetLemmatizer()
        return [lemmatizer.lemmatize(word) for word in words]
    
    # Combine text
    df['combined_text'] = (df['title'].apply(combine_text).fillna('') + ' ' +
                       df['description'].apply(combine_text).fillna('') + ' ' +
                       df['comments'].apply(combine_text))

    # Lower + removing trailing charsa
    df['combined_text'] = df['combined_text'].str.lower().str.strip()
    
    # Normalize university names
    df['combined_text'] = df['combined_text'].apply(normalize_university)

    # Tokenize + lemmatize
    df['lemmatized_tokens'] = df['combined_text'].apply(tokenize_and_lemmatize)

    # Return cleaned df
    return df

df = clean_text(df)

df.head()

,date_created,title,description,comments,upvotes,flair,university,combined_text,lemmatized_tokens
1,2024-11-08 03:24:09,[ Removed by Reddit ],[ Removed by Reddit on account of violating th...,"['Ik this man, he’s getting cooked. Maybe uoft...",2939,Discussion,UofT,"['ik this man, he’s getting cooked. maybe uoft...","[man, getting, cooked, maybe, uoft, anything, ..."
4,2025-05-14 00:11:02,University of Toronto Faculty Association vote...,NaN,"['Big W, honestly im surprised', 'W faculty', ...",2163,News,UofT,uoft faculty association votes to divest from ...,"[uoft, faculty, association, vote, divest, isr..."
13,2024-06-03 21:51:09,Just graduated at Convocation with encampment ...,"Just graduated and guess what, the encampment ...","[""that's so good to hear! i'm a huge supporter...",1615,Discussion,UofT,just graduated at convocation with encampment ...,"[graduated, convocation, encampment, present, ..."
22,2023-12-19 14:19:36,Is this MAT224 final average fr? (not my class...,NaN,['overconfident library vegetable dinosaurs ab...,1280,Courses,UofT,is this mat224 final average fr? (not my class...,"[final, average, fr, class, friend, sent, libr..."
27,2024-07-04 00:49:45,All that’s left of the encampment after todays...,Photo by @a1please on instagram,"['the grass is fucked 😭', ""Out of the loop, wh...",1271,News,UofT,all that’s left of the encampment after todays...,"[left, encampment, today, event, photo, instag..."


## Investigating top k words

In [37]:
from collections import Counter

def get_topk_words(df, k):
    '''
    Takes a df and returns top k words from tokens
    '''
    all_tokens = [token for row in df['lemmatized_tokens'] for token in row]

    # Count frequencies
    token_counts = Counter(all_tokens)

    # Get top K words
    top_k = token_counts.most_common(k)

    return top_k

# Get top 10 most common words across all posts
print(f'---All Posts---\n{get_topk_words(df, 10)}')

# Get top 10 most common words across flairs
for flair in df['flair'].unique():
    print(f'---{flair}---\n{get_topk_words(df[df['flair'] == flair], 10)}')


---All Posts---
[('people', 5321), ('like', 4291), ('student', 4116), ('get', 3449), ('year', 3143), ('would', 2922), ('one', 2864), ('time', 2728), ('think', 2630), ('know', 2322)]
---Discussion---
[('people', 3817), ('like', 3058), ('student', 2560), ('get', 2262), ('would', 1921), ('one', 1875), ('year', 1851), ('think', 1815), ('time', 1560), ('know', 1559)]
---News---
[('people', 699), ('student', 545), ('like', 378), ('get', 358), ('would', 343), ('one', 328), ('think', 313), ('uoft', 269), ('know', 269), ('http', 267)]
---Courses---
[('course', 336), ('student', 262), ('like', 218), ('class', 211), ('time', 149), ('one', 145), ('year', 144), ('prof', 143), ('would', 137), ('get', 133)]
---Academics---
[('student', 374), ('course', 234), ('exam', 194), ('get', 181), ('people', 172), ('like', 161), ('prof', 161), ('one', 143), ('would', 138), ('time', 129)]
---Advice---
[('year', 765), ('time', 598), ('get', 465), ('people', 402), ('like', 397), ('course', 369), ('take', 328), ('s

It seems all posts feature words that are probably common throughout the entire subreddit. Need to figure out a way to explore specific topics people are talking about..

Mutual Information (MI). If we treat our flair as a rough topic label, we can use MI to tell us information that is shared in the non-linear relationships between tokens and flairs.

In [ ]:
# 500 most common words
top_500 = get_topk_words(df, 500)

# Create matrix where columns have bool indicating whether specific post has a word 
mi_df = pd.DataFrame(df['flair'])

for word, _ in top_500:
    mi_df[word] = df['lemmatized_tokens'].apply(lambda tokens: int(word in tokens))

In [42]:
from sklearn.metrics import mutual_info_score

def getMI(topk, df, label_column = 'flair'):

    # Get flair names
    unique_labels = df[label_column].unique()

    # Placeholder df
    overalldf = pd.DataFrame()

    for flair in unique_labels:
        miscore = []
        label = df[label_column].copy()


        label[label != flair] = 0
        label[label == flair] = 1

        # Get MI for topk words in flair
        for word in topk:
            miscore.append([word[0]] + [mutual_info_score(label, df[word[0]])] + [flair])

        # Combine scores of all words for flair into df
        miscoredf = pd.DataFrame(miscore).sort_values(1, ascending = False)
        miscoredf.columns = ['word', 'mi', 'flair']
        overalldf = pd.concat([overalldf, miscoredf])

    return overalldf

In [45]:
mi_scores = getMI(top_500, mi_df)
mi_scores.set_index(['word']).groupby('flair')['mi'].nlargest(10).reset_index()

,flair,word,mi
0,Academics,ubc,0.026258
1,Academics,canada,0.024060
2,Academics,building,0.023334
3,Academics,big,0.021585
4,Academics,canadian,0.020141
5,Academics,probably,0.018747
6,Academics,anyone,0.018411
7,Academics,issue,0.018022
8,Academics,campus,0.017807
9,Academics,agree,0.017461


# Sentiment Analysis

In [52]:
from nltk.sentiment import SentimentIntensityAnalyzer

def evalsentences(sentences, to_df = False, columns = []):
    sid = SentimentIntensityAnalyzer()
    pdlist = []

    if to_df:
        for sentence in sentences:
            ss = sid.polarity_scores(sentence)
            pdlist.append([sentence] + [ss['compound']])
        df = pd.DataFrame(pdlist)
        df.columns = columns
        return df
    
    else:
        for sentrence in sentences:
            print('\n' + sentence)
            ss = sid.polarity_scores(sentence)
            for k in sorted(ss):
                print('{0}: {1}, '.format(k, ss[k], end = ''))
            print()

In [54]:
comments = df['combined_text']
vader_df = evalsentences(comments, to_df = True, columns = ['comments', 'vader'])

In [61]:
# See what comments are rated negative from VADER
pd.set_option('display.max_colwidth', 1000)
vader_df[vader_df['vader'] < 0]

,comments,vader
0,"['ik this man, he’s getting cooked. maybe uoft won’t do anything but it’s over for him and his social life esp w his gf\xa0', ""he apparently deleted his twitter account too, it's geniuenly over for bro"", 'i think the worst part of this has to be the fact that he peaked top 500 na in valorant.', ""'christ is king' jesus would def not approve of what you're saying buddy 😭"", 'pretty deranged of him to say that you and he probably got that from nick fuentes who is a self described nazi. report him for sure.', 'saw this posted on twitter and a bunch of girls came out of the woodwork w their own screenshots too', 'has anyone ever been suspended from uoft for something like this?', '[deleted]', 'as a montreal canadiens fan, we do not claim him. he’s on his own smh. \n\nwhat an ass.', 'some people really don’t know how to be a normal human being', 'average utsg cs experience', 'anyone got his linkedin ? 😂', ""commenting to let you know that i'm not even a uoft student, and the algorithm just...",-0.9978
1,"uoft faculty association votes to divest from israel ['big w, honestly im surprised', 'w faculty', 'honestly amazing to hear!', 'proud of my alma mater!', 'finally', 'noone in the planet will be free until palestine is free.\n\nacademics (people with great and undeniable intelligence) understand that.\n\n👏 👏 👏', ""finally. now let's see them actually do it"", 'it took them long enough bro', 'about time', 'well done, uoft! i’m really surprised, to be honest. but good on you!', 'i think both sides of the conflict are assholes for asking anyone to advocate for their beliefs that lead to murder. \n\nbut equally appalling is giving either side $ to promote their cultures.', 'this is misleading. they voted to recommend to the board that they vote to divest. plus it was barely over 50% of a vote that only like 4% of faculty engaged is. this affects and means nothing', 'wait we’re not funding genocide. ??', 'absurd how a canadian university has to take a vote in order to stop funding a fore...",-0.9594
2,"just graduated at convocation with encampment present just graduated and guess what, the encampment had little to no effect on the ceremony. cope mf’s who think the encampment was some huge negative for convocation. students were able to go on stage with keffiyeh and some went up with the palestinian flag and some with banners in support of the movement, and those students with the flag or banners probably got the loudest applause. overall great experience and no interruptions. \r\n\r\ncongrats to all my fellow grads!!! [""that's so good to hear! i'm a huge supporter of the encampment, but of course wouldn't want anyone's graduation to get severely impacted. i'm glad we can do both!"", 'echo chamber convocation.', '[deleted]', 'congratulations on graduating. now go be productive', ""a shame that there weren't lots of israeli flags represented on stage"", '[removed]', '[deleted]', 'gross', '[removed]', 'i’m so happy i declined my offer to that school 💀', 'technically the cultural and f...",-0.9983
4,"all that’s left of the encampment after todays events photo by @a1please on instagram ['the grass is fucked 😭', ""out of the loop, what were today's events?"", 'that’s hard', 'narrator voice: *they will not return*', 'iconic 👌\xa0\n\nand anyone complaining about the grass has clearly not spent a single winter semester at st george.', 'more people concerned with the state of the grass (which can grow back) than with the thousands killed in the genocide smh.', 'do these guys think this is a dc movie', ""itt: non uoft students so mad about uoft grass that they're willing to ignore the university's contributions to genocide and apartheid (through hidden investments).\n\n\ny'all need to get your priorities straight."", 'the most useless space ever. lets see what they do here, now that it’s “returned to the community” 😂', 'the poor grass 😔', 'now if only the school could bill back clean up and rehabilitation costs back to this g

In [62]:
# See what comments are rated negative from VADER
pd.set_option('display.max_colwidth', 1000)
vader_df[vader_df['vader'] > 0]

,comments,vader
3,"is this mat224 final average fr? (not my class, friend sent me) ['overconfident library vegetable dinosaurs aback voiceless market lush nutty summer\n\n *this post was mass deleted and anonymized with [redact](https://redact.dev)*', ""i want to talk to the one student who got an a. i can't even imagine how i'd feel looking at this graph if i were them"", ""can you share the full message please? i'm really curious what they said"", ""yikes‼️ i don't doubt it. reminds me of the phy132 final exam average in my first year..."", 'oh how good it must feel to be that one student who got a 90', 'high school grade inflation is starting to starting to create cracks in the system', 'yes, i’m in the course — mat224 at utm, there were some bonus points awarded.', 'can you send me the contact info of the kid who got an a? i am hiring.', 'why on earth were they so low?', 'u/mpaw976 comments?', 'had a similar result for year 2 calculus at sfu... class avg 28 %. the 28 was bell-curved into a 67 and that...",0.9987
5,"what is with the minecraft snow golem/s on campus?😂😂 woke up to this snow golem in the middle if the whitney hall quad 😂. apparently it was in queens park yesterday but i must’ve missed it. my big question is how the hell they moving it😂. this thing is made from wood and i’d say she’s pretty solid, i know they probably just move it on like a dolly or something but it’s still pretty cool lol. anyways whoever did it, it brightened up my day a little bit. has this happened before, is it pikachu dude related, i don’t know!?!😂😂 ['uoft biomes update. they naturally spawn now', 'golems move duh', 'this just made my day', 'it’s the whimsy of undefeated uoft student spirit', ""he's just trying to get to class"", 'hahaha brilliant', 'please stop building them im tired of shoveling everyday', 'is this photoshop?', 'i gotta find it next time', 'the zombies better look out', 'love this', ""no it's real - it was in kings college circle the other day""]",0.9788
6,"the current state of mat137 ['someone bring back alfonso’s smell test', ""i wore a well fitted mask to mat137 yesterday, could still smell the faint smell of armpit and unbrushed teeth.\n\ni know many people are living alone for the first time and are just simply ignorant of how to maintain hygiene so here are some tips:\n\n* **use a tongue scrapper** instead of using the bristles of your toothbrush to brush your tongue, gets rid of way more build up. (got mine on amazon)\n* **invest in a electric brush**, they clean so much better than a manual one, and faster too (used to have a lot cavities as a kid, but as soon as i got the electric my teeth improved significantly)\n* **brush twice day**, first in the morning (before or after breakfast, doesn't matter imo, just as long as you actually do it) and before you go to bed.\n* **floss!** gets rid of all that nasty plaque and old food that you would not be able to get with just brushing. you can either use floss picks or just the regul...",0.9984
7,"500+ students voted to cancel the final exam, and it worked [""as someone from utsc, the professor didn't show up for the last couple weeks of the course and released the lectures only a couple days ago. students in the class contacted the department and thus allowed students to vote whether to do a take home exam or void the exam entirely."", 'although i would probably vote yes, what about the students hoping to increase their grade through the exam?', 'media studies activities', 'any context? did they just vote to waive the final and it actually worked?', 'i don’t even go to this uni but 591/944 doesn’t seem like that strong of a majority. wouldn’t it make more sense to allow students to opt in to the final if they wanted, or opt out as they allowed here? what if students were banking on the final to bring their grade up?', 'what a dream', 'those 353 students who were failing must have been fucking pissed!!', 'hardest working media students:', 'so if you need to do well on 